In [ ]:
from wn import net

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch import nn

import pickle
import os

In [ ]:
# Load the tensorized data
with open("data/tensor_list.pkl", "rb") as f:
    input_data, y  = pickle.load(f)

# Load the interface
with open("data/match_interface.pkl", "rb") as f:
    match_interface = pickle.load(f)

In [ ]:
# This is what a size 16 batch should look like.
testo = {k: v[0:16, :] for k, v in input_data.items()}

In [ ]:
# Make an input layer
input_layer = net.TabularInputLayer(
    interface=match_interface,
    col_encoding_size=32,
    embedding_size=96,
    append_cls=True,
)

# Transformer encoder
tr = nn.TransformerEncoder(
    encoder_layer = nn.TransformerEncoderLayer(
        d_model=128,
        nhead=4,
        dim_feedforward=256,
        batch_first=True,
    ),
    num_layers=4,
)

# Ugly convenience layer
cls_selector = net.SelectCLSEncoding()

# Output layer
output_layers = net.OutputLayers(128, 3, 1)

whole_net = nn.Sequential(
    input_layer,
    tr,
    cls_selector,
    output_layers,
)

In [ ]:
whole_net(testo)